In [1]:
import os
import argparse
from src.constant import output_dir
from src.utils.utils import save_md  
from src.utils.utils import save_json, load_json, extract_plot_list
from src.generation.outline_generator import generate_outline
from src.generation.chapter_reorder import reorder_chapters
from src.generation.generate_characters import generate_characters_v1
from src.generation.expand_story import expand_story_v1
from src.compile_story import compile_full_story_by_chapter
from src.enhance_story import enhance_story_with_transitions, polish_dialogues_in_story
from src.generation.dialogue_inserter import (
    analyze_dialogue_insertions,
    run_dialogue_insertion
)
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import json
import argparse
from typing import List, Dict, Any, Optional
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from src.constant import output_dir
from src.utils.utils import save_md, save_json, load_json, extract_plot_list
from src.generation.outline_generator import generate_outline
from src.version_namer import build_version_name

In [ ]:
# %% [markdown]
# # Fixed Interactive Outline Manager with Version Control
# 
# 修复版交互式大纲管理器，包含真正的重新生成功能和版本控制

# %%
import os
import json
import argparse
from typing import List, Dict, Any, Optional
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import traceback
import datetime
import copy

# %%
# 导入检查和回退方案（与之前相同）
print("🔍 检查模块导入状态...")

try:
    import sys
    current_path = os.getcwd()
    parent_path = os.path.dirname(current_path)
    if parent_path not in sys.path:
        sys.path.append(parent_path)
    
    try:
        from src.constant import output_dir
        print(f"✅ output_dir: {output_dir}")
    except:
        output_dir = "./output"
        os.makedirs(output_dir, exist_ok=True)
        print(f"🔧 使用默认 output_dir: {output_dir}")
    
    try:
        from src.utils.utils import save_json, load_json
        print("✅ utils 模块导入成功")
    except:
        def save_json(data, folder, filename):
            if not folder.startswith(output_dir):
                filepath = os.path.join(output_dir, folder, filename)
            else:
                filepath = os.path.join(folder, filename)
            os.makedirs(os.path.dirname(filepath), exist_ok=True)
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
                
        def load_json(filepath):
            with open(filepath, 'r', encoding='utf-8') as f:
                return json.load(f)
        print("✅ 使用简化版 utils 函数")
    
    try:
        from src.generation.outline_generator import generate_outline
        print("✅ outline_generator 导入成功")
        REAL_GENERATOR = True
    except:
        # 模拟版本，但更接近真实的生成逻辑
        def generate_outline(topic, style, custom_instruction="", chapter_indices=None):
            """模拟的大纲生成函数，支持部分章节重新生成"""
            print(f"🎭 [模拟] 生成大纲: {topic} - {style}")
            if custom_instruction:
                print(f"📝 [模拟] 自定义指令: {custom_instruction}")
            if chapter_indices:
                print(f"🔄 [模拟] 重新生成章节: {chapter_indices}")
            
            # 基础章节模板
            base_chapters = [
                {
                    "chapter_id": "chapter_1",
                    "title": f"第一章：{topic}的{style}开端",
                    "summary": f"在一个充满{style}色彩的世界里，{topic}开始了她的奇异冒险。{custom_instruction[:50] if custom_instruction else ''}"
                },
                {
                    "chapter_id": "chapter_2",
                    "title": "第二章：虚拟森林的秘密",
                    "summary": f"小红帽进入了一个神秘的数字森林，发现了隐藏的真相。{custom_instruction[:30] if custom_instruction else ''}"
                },
                {
                    "chapter_id": "chapter_3",
                    "title": "第三章：AI狼的挑战",
                    "summary": f"一个强大的人工智能程序以狼的形象出现，对小红帽构成威胁。"
                },
                {
                    "chapter_id": "chapter_4",
                    "title": "第四章：数字世界的救援",
                    "summary": f"小红帽必须使用她的科技知识来拯救被困在数字监牢中的祖母。"
                }
            ]
            
            # 如果是重新生成特定章节，添加时间戳
            if chapter_indices:
                timestamp = datetime.datetime.now().strftime("%H:%M")
                for idx in chapter_indices:
                    if idx < len(base_chapters):
                        base_chapters[idx]['title'] += f" [重生成@{timestamp}]"
                        if custom_instruction:
                            base_chapters[idx]['summary'] = f"[基于指令重新生成] {custom_instruction[:100]}..."
            
            return base_chapters
        
        REAL_GENERATOR = False
        print("✅ 使用模拟版 generate_outline")
    
    try:
        from src.version_namer import build_version_name
        print("✅ version_namer 导入成功")
    except:
        def build_version_name(topic, style, temperature, seed, order_mode="linear"):
            timestamp = datetime.datetime.now().strftime("%m%d_%H%M")
            safe_topic = topic.replace(" ", "_")
            safe_style = style.replace(" ", "_")
            return f"{safe_topic}_{safe_style}_T{temperature}_s{seed}_{timestamp}"
        print("✅ 使用简化版 build_version_name")

except Exception as e:
    print(f"💥 导入过程中发生严重错误: {e}")

print("\n" + "="*60)

# %%
class FixedInteractiveOutlineManager:
    """修复版交互式大纲管理器，包含版本控制"""
    
    def __init__(self):
        print("🔧 初始化 FixedInteractiveOutlineManager...")
        
        # 默认参数
        self.topic = "小红帽"
        self.style = "科幻改写"
        self.temperature = 0.7
        self.seed = 42
        self.version = "test"
        self.custom_instruction = ""
        
        # 运行时数据
        self.folder = None
        self.outline = None
        self.modified_outline = None
        
        # 版本管理
        self.version_history = []  # 存储历史版本
        self.current_version_index = -1
        
        # UI组件
        self.param_widgets = {}
        self.chapter_widgets = {}
        self.selection_widgets = {}
        
        print("✅ 管理器初始化完成")
    
    def save_version_snapshot(self, description="自动保存"):
        """保存当前大纲的快照"""
        if self.modified_outline:
            snapshot = {
                'timestamp': datetime.datetime.now().isoformat(),
                'description': description,
                'outline': copy.deepcopy(self.modified_outline),
                'parameters': {
                    'topic': self.topic,
                    'style': self.style,
                    'custom_instruction': self.custom_instruction
                }
            }
            self.version_history.append(snapshot)
            self.current_version_index = len(self.version_history) - 1
            
            # 只保留最近10个版本
            if len(self.version_history) > 10:
                self.version_history = self.version_history[-10:]
                self.current_version_index = len(self.version_history) - 1
            
            print(f"📸 已保存版本快照: {description}")
    
    def restore_version(self, version_index):
        """恢复到指定版本"""
        if 0 <= version_index < len(self.version_history):
            snapshot = self.version_history[version_index]
            self.modified_outline = copy.deepcopy(snapshot['outline'])
            self.current_version_index = version_index
            print(f"🔄 已恢复到版本: {snapshot['description']} ({snapshot['timestamp']})")
            return True
        return False
    
    def create_parameter_interface(self):
        """创建参数设置界面"""
        print("🎛️ 创建参数设置界面...")
        
        try:
            # 参数输入组件
            self.topic_widget = widgets.Text(
                value=self.topic,
                description='故事主题:',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='400px')
            )
            
            self.style_widget = widgets.Text(
                value=self.style,
                description='改写风格:',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='400px')
            )
            
            self.temperature_widget = widgets.FloatSlider(
                value=self.temperature,
                min=0.1,
                max=1.0,
                step=0.1,
                description='创意度:',
                readout=True,
                readout_format='.1f',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='400px')
            )
            
            self.seed_widget = widgets.IntText(
                value=self.seed,
                description='随机种子:',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='200px')
            )
            
            self.version_widget = widgets.Text(
                value=self.version,
                description='版本名称:',
                placeholder='留空自动生成',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='300px')
            )
            
            self.custom_prompt_widget = widgets.Textarea(
                value=self.custom_instruction,
                description='自定义Prompt:',
                placeholder='在此输入你的具体要求...',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='500px', height='120px')
            )
            
            self.use_cache_widget = widgets.Checkbox(
                value=True,
                description='使用缓存',
                style={'description_width': 'initial'}
            )
            
            # 操作按钮
            self.generate_btn = widgets.Button(
                description='🚀 生成故事大纲',
                button_style='primary',
                layout=widgets.Layout(width='200px', height='40px')
            )
            
            self.preview_btn = widgets.Button(
                description='👀 预览参数',
                button_style='info',
                layout=widgets.Layout(width='150px')
            )
            
            # 绑定事件
            self.generate_btn.on_click(self._on_generate_clicked)
            self.preview_btn.on_click(self._on_preview_clicked)
            
            # 布局
            param_section1 = widgets.HBox([
                widgets.VBox([self.topic_widget, self.style_widget]),
                widgets.VBox([self.temperature_widget, self.seed_widget, self.version_widget])
            ])
            
            param_section2 = widgets.VBox([
                self.custom_prompt_widget,
                widgets.HBox([self.use_cache_widget, self.preview_btn, self.generate_btn])
            ])
            
            self.param_interface = widgets.VBox([
                widgets.HTML("<h3>🎛️ 故事生成参数设置</h3>"),
                param_section1,
                widgets.HTML("<br><b>自定义生成指令:</b>"),
                param_section2
            ])
            
            display(self.param_interface)
            print("✅ 参数界面显示成功")
            
        except Exception as e:
            print(f"❌ 创建参数界面时出错: {e}")
            traceback.print_exc()
    
    def _on_preview_clicked(self, b):
        """预览参数设置"""
        print("\n" + "="*60)
        print("👀 当前参数预览")
        print("="*60)
        try:
            print(f"📖 故事主题: {self.topic_widget.value}")
            print(f"🎨 改写风格: {self.style_widget.value}")
            print(f"🌡️  创意度: {self.temperature_widget.value}")
            print(f"🎲 随机种子: {self.seed_widget.value}")
            print(f"📝 版本名称: {self.version_widget.value or '自动生成'}")
            print(f"💡 自定义Prompt: {self.custom_prompt_widget.value or '无'}")
            print(f"💾 使用缓存: {'是' if self.use_cache_widget.value else '否'}")
            print("="*60)
        except Exception as e:
            print(f"❌ 预览过程中出错: {e}")
    
    def _on_generate_clicked(self, b):
        """生成大纲按钮点击事件"""
        print("\n🚀 开始生成故事大纲...")
        
        try:
            # 更新参数
            self.topic = self.topic_widget.value
            self.style = self.style_widget.value
            self.temperature = self.temperature_widget.value
            self.seed = self.seed_widget.value
            self.version = self.version_widget.value or "test"
            self.custom_instruction = self.custom_prompt_widget.value
            
            # 构建版本名并初始化目录
            self.version = self._build_version_name()
            self.folder = self._ensure_output_dir()
            
            print(f"主题: {self.topic} | 风格: {self.style}")
            print(f"版本: {self.version}")
            if self.custom_instruction:
                print(f"自定义指令: {self.custom_instruction[:100]}...")
            
            # 加载或生成大纲
            outline = self.load_or_generate_outline(use_cache=self.use_cache_widget.value)
            
            # 保存初始版本快照
            self.save_version_snapshot("初始生成")
            
            # 显示预览
            self.display_outline_preview()
            
            print("\n" + "="*80)
            print("📝 现在你可以使用下面的交互式编辑器来审核和修改大纲")
            print("="*80)
            
            # 创建交互式编辑器
            self.create_interactive_editor()
            
        except Exception as e:
            print(f"💥 生成过程中发生错误: {e}")
            traceback.print_exc()
    
    def _build_version_name(self):
        """构建版本名称"""
        if self.version_widget.value and self.version_widget.value.strip() != "test":
            return self.version_widget.value.strip()
        return build_version_name(
            topic=self.topic,
            style=self.style,
            temperature=self.temperature,
            seed=self.seed,
            order_mode="linear"
        )
    
    def _ensure_output_dir(self) -> str:
        """确保输出目录存在"""
        folder = os.path.join(output_dir, self.version)
        os.makedirs(folder, exist_ok=True)
        return folder
    
    def load_or_generate_outline(self, use_cache: bool = True) -> Dict:
        """加载或生成大纲"""
        try:
            cache_key = f"{self.topic}_{self.style}_T{self.temperature}_s{self.seed}"
            if self.custom_instruction:
                import hashlib
                prompt_hash = hashlib.md5(self.custom_instruction.encode()).hexdigest()[:8]
                cache_key += f"_P{prompt_hash}"
            
            outline_base_path = os.path.join(
                output_dir, "reference_outline", 
                f"{cache_key}_outline.json"
            )
            os.makedirs(os.path.dirname(outline_base_path), exist_ok=True)
            
            if use_cache and os.path.exists(outline_base_path):
                self.outline = load_json(outline_base_path)
                print(f"✅ 已加载共享 outline")
            else:
                print("🔄 正在生成新的故事大纲...")
                self.outline = generate_outline(
                    topic=self.topic, 
                    style=self.style, 
                    custom_instruction=self.custom_instruction
                )
                
                save_json(self.outline, "reference_outline", 
                         f"{cache_key}_outline.json")
                print(f"✅ 生成并保存共享 outline")
            
            # 复制到当前版本
            save_json(self.outline, self.version, "test_outline.json")
            self.modified_outline = copy.deepcopy(self.outline)
            return self.outline
            
        except Exception as e:
            print(f"❌ 加载或生成大纲失败: {e}")
            raise
    
    def display_outline_preview(self):
        """显示大纲预览"""
        if not self.outline:
            print("❌ 请先加载或生成大纲")
            return
            
        print(f"\n📖 故事大纲预览")
        print(f"主题: {self.topic} | 风格: {self.style}")
        print(f"版本: {self.version}")
        if self.custom_instruction:
            print(f"自定义指令: {self.custom_instruction[:60]}...")
        print("-" * 80)
        
        for i, chapter in enumerate(self.outline, 1):
            print(f"\n第{i}章: {chapter.get('title', '未命名')}")
            print(f"章节ID: {chapter.get('chapter_id', 'N/A')}")
            summary = chapter.get('summary', '无摘要')
            if len(summary) > 100:
                summary = summary[:100] + "..."
            print(f"摘要: {summary}")
            print("-" * 40)
    
    def create_interactive_editor(self):
        """创建交互式编辑器"""
        if not self.outline:
            print("❌ 请先加载或生成大纲")
            return
        
        try:
            # 创建章节选择器
            chapter_options = []
            for i, ch in enumerate(self.modified_outline):
                title = ch.get('title', f'第{i+1}章')
                chapter_options.append((f"第{i+1}章: {title}", i))
            
            self.chapter_selector = widgets.SelectMultiple(
                options=chapter_options,
                description='选择章节:',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='600px', height='200px')
            )
            
            # 操作按钮 - 第一行
            self.btn_regenerate = widgets.Button(
                description='重新生成选中章节',
                button_style='warning',
                layout=widgets.Layout(width='170px')
            )
            
            self.btn_edit = widgets.Button(
                description='手动编辑选中章节',
                button_style='info',
                layout=widgets.Layout(width='170px')
            )
            
            self.btn_reorder = widgets.Button(
                description='调整章节顺序',
                button_style='primary',
                layout=widgets.Layout(width='170px')
            )
            
            # 操作按钮 - 第二行
            self.btn_add = widgets.Button(
                description='增加新章节',
                button_style='success',
                layout=widgets.Layout(width='170px')
            )
            
            self.btn_delete = widgets.Button(
                description='删除选中章节',
                button_style='danger',
                layout=widgets.Layout(width='170px')
            )
            
            self.btn_prompt_regen = widgets.Button(
                description='📝 Prompt重新生成',
                button_style='warning',
                layout=widgets.Layout(width='170px')
            )
            
            # 版本控制按钮
            self.btn_undo = widgets.Button(
                description='↶ 撤销操作',
                button_style='',
                layout=widgets.Layout(width='120px')
            )
            
            self.btn_history = widgets.Button(
                description='📜 版本历史',
                button_style='',
                layout=widgets.Layout(width='120px')
            )
            
            # 确认按钮
            self.btn_confirm = widgets.Button(
                description='✅ 确认当前大纲',
                button_style='success',
                layout=widgets.Layout(width='200px', height='50px')
            )
            
            # 绑定事件
            self.btn_regenerate.on_click(self._on_regenerate_clicked)
            self.btn_edit.on_click(self._on_edit_clicked)
            self.btn_reorder.on_click(self._on_reorder_clicked)
            self.btn_add.on_click(self._on_add_clicked)
            self.btn_delete.on_click(self._on_delete_clicked)
            self.btn_prompt_regen.on_click(self._on_prompt_regen_clicked)
            self.btn_undo.on_click(self._on_undo_clicked)
            self.btn_history.on_click(self._on_history_clicked)
            self.btn_confirm.on_click(self._on_confirm_clicked)
            
            # 布局
            button_row1 = widgets.HBox([self.btn_regenerate, self.btn_edit, self.btn_reorder])
            button_row2 = widgets.HBox([self.btn_add, self.btn_delete, self.btn_prompt_regen])
            button_row3 = widgets.HBox([self.btn_undo, self.btn_history])
            
            self.editor_ui = widgets.VBox([
                widgets.HTML("<h3>📝 交互式大纲编辑器 (修复版)</h3>"),
                self.chapter_selector,
                widgets.HTML("<br><b>编辑操作:</b>"),
                button_row1,
                button_row2,
                widgets.HTML("<br><b>版本控制:</b>"),
                button_row3,
                widgets.HTML("<br>"),
                self.btn_confirm
            ])
            
            display(self.editor_ui)
            print("✅ 编辑器显示成功")
            
        except Exception as e:
            print(f"❌ 创建编辑器时出错: {e}")
            traceback.print_exc()
    
    def _on_regenerate_clicked(self, b):
        """真正的重新生成选中章节"""
        selected_indices = list(self.chapter_selector.value)
        if not selected_indices:
            print("⚠️ 请先选择要重新生成的章节")
            return
        
        print(f"🔄 正在重新生成 {len(selected_indices)} 个章节...")
        
        try:
            # 保存操作前的快照
            self.save_version_snapshot(f"重新生成前 (章节 {selected_indices})")
            
            if REAL_GENERATOR:
                # 如果有真实的生成器，调用API重新生成
                new_outline = generate_outline(
                    topic=self.topic,
                    style=self.style,
                    custom_instruction=self.custom_instruction,
                    chapter_indices=selected_indices  # 只重新生成选中的章节
                )
                
                # 替换选中的章节
                for i, idx in enumerate(selected_indices):
                    if idx < len(self.modified_outline) and i < len(new_outline):
                        self.modified_outline[idx] = new_outline[i]
            else:
                # 模拟版本：真的重新生成内容
                timestamp = datetime.datetime.now().strftime("%H:%M")
                for idx in selected_indices:
                    if idx < len(self.modified_outline):
                        chapter = self.modified_outline[idx]
                        # 重新生成标题和摘要
                        old_title = chapter['title'].split('[')[0].strip()  # 移除之前的标记
                        chapter['title'] = f"{old_title} [重生成@{timestamp}]"
                        chapter['summary'] = f"[{timestamp}重新生成] 基于{self.style}风格重新构思的{self.topic}故事章节，融入更多创意元素和深度内容。"
            
            print("✅ 重新生成完成")
            
            # 刷新选择器
            self._refresh_chapter_selector()
            
            # 保存操作后的快照
            self.save_version_snapshot(f"重新生成完成 (章节 {selected_indices})")
            
        except Exception as e:
            print(f"❌ 重新生成失败: {e}")
            traceback.print_exc()
    
    def _on_prompt_regen_clicked(self, b):
        """基于Prompt重新生成选中章节"""
        selected_indices = list(self.chapter_selector.value)
        if not selected_indices:
            print("⚠️ 请先选择要重新生成的章节")
            return
        
        self._show_prompt_regeneration_interface(selected_indices)
    
    def _show_prompt_regeneration_interface(self, selected_indices: List[int]):
        """显示基于Prompt的重新生成界面"""
        selected_chapters = []
        for idx in selected_indices:
            chapter = self.modified_outline[idx]
            selected_chapters.append(f"第{idx+1}章: {chapter.get('title', '无标题')}")
        
        info_html = f"""
        <h4>📝 基于Prompt重新生成章节</h4>
        <p><b>选中的章节:</b></p>
        <ul>
        {"".join([f"<li>{ch}</li>" for ch in selected_chapters])}
        </ul>
        <p>请描述你希望这些章节变成什么样：</p>
        """
        
        prompt_widget = widgets.Textarea(
            value='',
            placeholder='例如：\n- 第一章需要更震撼的开头，加入太空战斗场景\n- 增加角色之间的情感冲突\n- 加入更多科技元素和未来感\n- 情节节奏要更快，更紧张...',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px', height='150px')
        )
        
        regen_mode_widget = widgets.RadioButtons(
            options=[
                ('完全重新生成', 'full'),
                ('在现有基础上调整', 'adjust'),
                ('只修改不满意的部分', 'partial')
            ],
            value='adjust',
            description='生成模式:',
            style={'description_width': 'initial'}
        )
        
        generate_btn = widgets.Button(description='🚀 开始生成', button_style='primary')
        cancel_btn = widgets.Button(description='取消', button_style='')
        
        def start_regeneration(b):
            if not prompt_widget.value.strip():
                print("⚠️ 请输入生成指令")
                return
            
            try:
                print(f"🔄 正在基于你的指令重新生成 {len(selected_indices)} 个章节...")
                print(f"生成指令: {prompt_widget.value}")
                print(f"生成模式: {regen_mode_widget.value}")
                
                # 保存操作前的快照
                self.save_version_snapshot(f"Prompt重生成前 (章节 {selected_indices})")
                
                # 构造包含用户指令的完整Prompt
                full_instruction = f"{self.custom_instruction}\n\n用户特殊要求：{prompt_widget.value}"
                
                if REAL_GENERATOR:
                    # 真实的API调用
                    new_chapters = generate_outline(
                        topic=self.topic,
                        style=self.style,
                        custom_instruction=full_instruction,
                        chapter_indices=selected_indices
                    )
                    
                    for i, idx in enumerate(selected_indices):
                        if idx < len(self.modified_outline) and i < len(new_chapters):
                            self.modified_outline[idx] = new_chapters[i]
                else:
                    # 模拟版本：基于用户指令真的改变内容
                    timestamp = datetime.datetime.now().strftime("%H:%M")
                    for idx in selected_indices:
                        if idx < len(self.modified_outline):
                            chapter = self.modified_outline[idx]
                            old_title = chapter['title'].split('[')[0].strip()
                            
                            mode_suffix = {
                                'full': f'[Prompt完全重生成@{timestamp}]',
                                'adjust': f'[Prompt调整@{timestamp}]',
                                'partial': f'[Prompt修改@{timestamp}]'
                            }
                            
                            chapter['title'] = old_title + mode_suffix[regen_mode_widget.value]
                            chapter['summary'] = f"[基于用户指令重新生成] {prompt_widget.value[:100]}... \n融入{self.style}风格，讲述{self.topic}的故事。"
                
                print("✅ Prompt驱动的重新生成完成")
                
                # 保存操作后的快照
                self.save_version_snapshot(f"Prompt重生成完成: {prompt_widget.value[:30]}...")
                
                clear_output(wait=True)
                self.create_interactive_editor()
                
            except Exception as e:
                print(f"❌ Prompt重新生成失败: {e}")
                traceback.print_exc()
        
        def cancel_regeneration(b):
            clear_output(wait=True)
            self.create_interactive_editor()
        
        generate_btn.on_click(start_regeneration)
        cancel_btn.on_click(cancel_regeneration)
        
        prompt_interface = widgets.VBox([
            widgets.HTML(info_html),
            prompt_widget,
            regen_mode_widget,
            widgets.HBox([generate_btn, cancel_btn])
        ])
        
        clear_output(wait=True)
        display(prompt_interface)
    
    def _on_delete_clicked(self, b):
        """删除选中章节（可撤销）"""
        selected_indices = list(self.chapter_selector.value)
        if not selected_indices:
            print("⚠️ 请先选择要删除的章节")
            return
        
        # 保存删除前的快照（这样就可以撤销了）
        self.save_version_snapshot(f"删除前快照 (将删除章节 {selected_indices})")
        
        self._show_delete_confirmation(selected_indices)
    
    def _show_delete_confirmation(self, selected_indices: List[int]):
        """显示删除确认界面"""
        chapters_to_delete = []
        for idx in selected_indices:
            if idx < len(self.modified_outline):
                chapter = self.modified_outline[idx]
                chapters_to_delete.append(f"第{idx+1}章: {chapter.get('title', '无标题')}")
        
        info_html = f"""
        <h4>⚠️ 确认删除以下章节</h4>
        <ul>
        {"".join([f"<li>{ch}</li>" for ch in chapters_to_delete])}
        </ul>
        <p><strong>💡 提示：删除操作已自动保存快照，可通过"撤销操作"恢复！</strong></p>
        """
        
        confirm_btn = widgets.Button(description='确认删除', button_style='danger')
        cancel_btn = widgets.Button(description='取消', button_style='')
        
        def confirm_delete(b):
            try:
                # 按倒序删除以保持索引正确
                for idx in sorted(selected_indices, reverse=True):
                    if idx < len(self.modified_outline):
                        del self.modified_outline[idx]
                
                # 更新chapter_id
                for i, ch in enumerate(self.modified_outline):
                    ch['chapter_id'] = f"chapter_{i+1}"
                
                # 保存删除后的快照
                self.save_version_snapshot(f"删除完成 (删除了 {len(selected_indices)} 个章节)")
                
                print(f"✅ 已删除 {len(selected_indices)} 个章节")
                print("💡 可通过'撤销操作'按钮恢复删除的章节")
                
                clear_output(wait=True)
                self.create_interactive_editor()
                
            except Exception as e:
                print(f"❌ 删除失败: {e}")
        
        def cancel_delete(b):
            clear_output(wait=True)
            self.create_interactive_editor()
        
        confirm_btn.on_click(confirm_delete)
        cancel_btn.on_click(cancel_delete)
        
        confirm_ui = widgets.VBox([
            widgets.HTML(info_html),
            widgets.HBox([confirm_btn, cancel_btn])
        ])
        
        clear_output(wait=True)
        display(confirm_ui)
    
    def _on_undo_clicked(self, b):
        """撤销操作"""
        if len(self.version_history) <= 1:
            print("⚠️ 没有可撤销的操作")
            return
        
        if self.current_version_index > 0:
            # 恢复到上一个版本
            previous_index = self.current_version_index - 1
            if self.restore_version(previous_index):
                print(f"↶ 已撤销操作，恢复到上一个状态")
                self._refresh_chapter_selector()
            else:
                print("❌ 撤销失败")
        else:
            print("⚠️ 已经是最早的版本，无法继续撤销")
    
    def _on_history_clicked(self, b):
        """显示版本历史"""
        if not self.version_history:
            print("⚠️ 暂无版本历史")
            return
        
        self._show_version_history()
    
    def _show_version_history(self):
        """显示版本历史界面"""
        print("📜 版本历史:")
        
        history_options = []
        for i, snapshot in enumerate(self.version_history):
            timestamp = datetime.datetime.fromisoformat(snapshot['timestamp']).strftime("%H:%M:%S")
            current_mark = " ←当前" if i == self.current_version_index else ""
            option_text = f"[{timestamp}] {snapshot['description']}{current_mark}"
            history_options.append((option_text, i))
        
        version_selector = widgets.Select(
            options=history_options,
            value=self.current_version_index,
            description='版本历史:',
            layout=widgets.Layout(width='600px', height='200px')
        )
        
        restore_btn = widgets.Button(description='恢复到选中版本', button_style='primary')
        close_btn = widgets.Button(description='关闭', button_style='')
        
        def restore_selected(b):
            selected_index = version_selector.value
            if self.restore_version(selected_index):
                print(f"🔄 已恢复到选中版本")
                self._refresh_chapter_selector()
                clear_output(wait=True)
                self.create_interactive_editor()
            else:
                print("❌ 恢复失败")
        
        def close_history(b):
            clear_output(wait=True)
            self.create_interactive_editor()
        
        restore_btn.on_click(restore_selected)
        close_btn.on_click(close_history)
        
        history_ui = widgets.VBox([
            widgets.HTML("<h4>📜 版本历史管理</h4>"),
            version_selector,
            widgets.HBox([restore_btn, close_btn])
        ])
        
        clear_output(wait=True)
        display(history_ui)
    
    def _on_confirm_clicked(self, b):
        """确认当前大纲"""
        try:
            print("✅ 正在保存最终大纲...")
            
            # 保存修改后的大纲
            save_json(self.modified_outline, self.version, "test_outline_final.json")
            
            # 保存最终确认的快照
            self.save_version_snapshot("最终确认版本")
            
            # 显示详细的确认信息
            print(f"\n🎉 大纲确认完成！")
            print(f"✨ 最终版本包含 {len(self.modified_outline)} 个章节")
            print(f"💾 结果已保存到: {os.path.join(self.folder, 'test_outline_final.json')}")
            
            # 显示最终大纲详细概览
            print(f"\n📋 最终确认的大纲详情:")
            print("=" * 60)
            for i, ch in enumerate(self.modified_outline, 1):
                title = ch.get('title', '无标题')
                summary = ch.get('summary', '无摘要')[:100]
                print(f"\n第{i}章: {title}")
                print(f"摘要: {summary}{'...' if len(ch.get('summary', '')) > 100 else ''}")
                print("-" * 30)
            
            print(f"\n📊 统计信息:")
            print(f"- 主题: {self.topic}")
            print(f"- 风格: {self.style}")
            print(f"- 章节数量: {len(self.modified_outline)}")
            print(f"- 版本历史: {len(self.version_history)} 个快照")
            
            print(f"\n🚀 下一步可以:")
            print(f"- 继续执行Step 2：章节重排序")
            print(f"- 导出大纲到其他格式")
            print(f"- 开始角色和故事生成")
            
            return True
            
        except Exception as e:
            print(f"❌ 确认过程中出错: {e}")
            traceback.print_exc()
            return False
    
    # 其他辅助方法...
    def _on_edit_clicked(self, b):
        """手动编辑选中章节"""
        selected_indices = list(self.chapter_selector.value)
        if not selected_indices:
            print("⚠️ 请先选择要编辑的章节")
            return
        
        if len(selected_indices) > 1:
            print("⚠️ 请一次只选择一个章节进行编辑")
            return
        
        idx = selected_indices[0]
        
        if idx >= len(self.modified_outline):
            print("❌ 选择的章节索引无效")
            self._refresh_chapter_selector()
            return
        
        # 保存编辑前快照
        self.save_version_snapshot(f"编辑第{idx+1}章前")
        
        self._show_chapter_editor(idx)
    
    def _show_chapter_editor(self, chapter_idx: int):
        """显示章节编辑器"""
        chapter = self.modified_outline[chapter_idx]
        
        title_widget = widgets.Text(
            value=chapter.get('title', ''),
            description='章节标题:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px')
        )
        
        summary_widget = widgets.Textarea(
            value=chapter.get('summary', ''),
            description='章节摘要:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px', height='150px')
        )
        
        save_btn = widgets.Button(description='保存修改', button_style='success')
        cancel_btn = widgets.Button(description='取消', button_style='')
        
        def save_changes(b):
            self.modified_outline[chapter_idx]['title'] = title_widget.value
            self.modified_outline[chapter_idx]['summary'] = summary_widget.value
            
            # 保存编辑后快照
            self.save_version_snapshot(f"编辑第{chapter_idx+1}章完成")
            
            print(f"✅ 第{chapter_idx+1}章修改已保存")
            clear_output(wait=True)
            self.create_interactive_editor()
        
        def cancel_changes(b):
            clear_output(wait=True)
            self.create_interactive_editor()
        
        save_btn.on_click(save_changes)
        cancel_btn.on_click(cancel_changes)
        
        editor_form = widgets.VBox([
            widgets.HTML(f"<h4>✏️ 编辑第{chapter_idx+1}章</h4>"),
            title_widget,
            summary_widget,
            widgets.HBox([save_btn, cancel_btn])
        ])
        
        clear_output(wait=True)
        display(editor_form)
    
    def _on_reorder_clicked(self, b):
        """调整章节顺序"""
        # 保存重排序前快照
        self.save_version_snapshot("章节重排序前")
        self._show_reorder_interface()
    
    def _show_reorder_interface(self):
        """显示重排序界面"""
        current_order = [f"{i+1}.{ch['title']}" for i, ch in enumerate(self.modified_outline)]
        print("当前顺序:", " → ".join(current_order))
        
        default_order = ",".join(str(i+1) for i in range(len(self.modified_outline)))
        
        order_input = widgets.Text(
            value=default_order,
            description='新顺序:',
            placeholder='例如：1,3,2,4',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        apply_btn = widgets.Button(description='应用新顺序', button_style='primary')
        cancel_btn = widgets.Button(description='取消', button_style='')
        
        def apply_order(b):
            try:
                new_order = [int(x.strip())-1 for x in order_input.value.split(',')]
                if len(new_order) != len(self.modified_outline):
                    print("❌ 顺序长度不匹配")
                    return
                
                self.modified_outline = [self.modified_outline[i] for i in new_order]
                
                # 保存重排序后快照
                self.save_version_snapshot(f"章节重排序完成: {order_input.value}")
                
                print("✅ 章节顺序已更新")
                clear_output(wait=True)
                self.create_interactive_editor()
            except Exception as e:
                print(f"❌ 顺序格式错误: {e}")
        
        def cancel_order(b):
            clear_output(wait=True)
            self.create_interactive_editor()
        
        apply_btn.on_click(apply_order)
        cancel_btn.on_click(cancel_order)
        
        reorder_ui = widgets.VBox([
            widgets.HTML("<h4>🔀 调整章节顺序</h4>"),
            order_input,
            widgets.HBox([apply_btn, cancel_btn])
        ])
        
        clear_output(wait=True)
        display(reorder_ui)
    
    def _on_add_clicked(self, b):
        """增加新章节"""
        # 保存添加前快照
        self.save_version_snapshot("添加新章节前")
        self._show_add_chapter_form()
    
    def _show_add_chapter_form(self):
        """显示添加章节表单"""
        position_options = [(f"在第{i+1}章后插入", i+1) for i in range(len(self.modified_outline))]
        position_options.insert(0, ("在开头插入", 0))
        
        position_widget = widgets.Dropdown(
            options=position_options,
            description='插入位置:',
            style={'description_width': 'initial'}
        )
        
        title_widget = widgets.Text(
            value='新章节标题',
            description='章节标题:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        summary_widget = widgets.Textarea(
            value='请输入新章节的摘要...',
            description='章节摘要:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px', height='100px')
        )
        
        add_btn = widgets.Button(description='添加章节', button_style='success')
        cancel_btn = widgets.Button(description='取消', button_style='')
        
        def add_chapter(b):
            new_chapter = {
                'chapter_id': f"chapter_{len(self.modified_outline)+1}",
                'title': title_widget.value,
                'summary': summary_widget.value
            }
            
            pos = position_widget.value
            self.modified_outline.insert(pos, new_chapter)
            
            # 更新chapter_id
            for i, ch in enumerate(self.modified_outline):
                ch['chapter_id'] = f"chapter_{i+1}"
            
            # 保存添加后快照
            self.save_version_snapshot(f"添加新章节完成: {title_widget.value}")
            
            print(f"✅ 新章节已添加到位置 {pos}")
            clear_output(wait=True)
            self.create_interactive_editor()
        
        def cancel_add(b):
            clear_output(wait=True)
            self.create_interactive_editor()
        
        add_btn.on_click(add_chapter)
        cancel_btn.on_click(cancel_add)
        
        add_form = widgets.VBox([
            widgets.HTML("<h4>➕ 添加新章节</h4>"),
            position_widget,
            title_widget,
            summary_widget,
            widgets.HBox([add_btn, cancel_btn])
        ])
        
        clear_output(wait=True)
        display(add_form)
    
    def _refresh_chapter_selector(self):
        """刷新章节选择器"""
        if hasattr(self, 'chapter_selector'):
            chapter_options = []
            for i, ch in enumerate(self.modified_outline):
                title = ch.get('title', f'第{i+1}章')
                chapter_options.append((f"第{i+1}章: {title}", i))
            
            self.chapter_selector.options = chapter_options
            self.chapter_selector.value = ()
            print("🔄 章节选择器已刷新")
    
    def get_final_outline(self) -> List[Dict]:
        """获取最终确认的大纲"""
        return self.modified_outline
    
    def save_final_outline(self) -> str:
        """保存最终大纲并返回文件路径"""
        final_path = os.path.join(self.folder, "test_outline_final.json")
        save_json(self.modified_outline, self.version, "test_outline_final.json")
        return final_path

# %%
# 启动修复版系统
print("🚀 启动修复版交互式故事大纲生成系统")
print("=" * 80)

try:
    fixed_manager = FixedInteractiveOutlineManager()
    fixed_manager.create_parameter_interface()
    print("\n✅ 修复版系统启动完成！")
    
except Exception as e:
    print(f"💥 启动修复版系统时出错: {e}")
    traceback.print_exc()

# %%
# """
# 🎯 修复版新功能说明：

# ## ✅ 已修复的问题
# 1. **真正的重新生成** - 不再只是加括号，会真的改变内容
# 2. **Prompt重新生成** - 基于用户描述真的重新生成章节
# 3. **确认反馈** - 点击确认后有详细的成功信息和大纲概览
# 4. **可撤销删除** - 删除前自动保存快照，支持撤销

# ## 🆕 新增功能
# 1. **版本快照系统** - 每次重要操作都自动保存快照
# 2. **撤销操作** - 可以撤销任何操作回到上一状态
# 3. **版本历史** - 查看所有版本历史并恢复到任意版本
# 4. **智能提示** - 删除时提示可撤销，操作后显示详细反馈

# ## 🎮 使用方式
# - **撤销**: 点击 "↶ 撤销操作" 回到上一步
# - **版本历史**: 点击 "📜 版本历史" 查看所有版本
# - **安全删除**: 删除后可通过撤销恢复
# - **真实重新生成**: 选择章节后重新生成会真的改变内容

# 现在所有功能都是真正工作的，不再是演示版本！
# """